In [2]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import re
import networkx as nx
import datetime as dt
import time

from cleaning_functions import *
from cleaners import *

%matplotlib notebook

In [3]:
df = pd.read_csv('cleaned_podcasts.csv', sep='\t', index_col=0)
df

,date,duration,episode,guests,hosts,podcast,title
0,2018-03-20 22:29:51,10603,1094,Brian Redban,Joe Rogan,The Joe Rogan Experience,#1094 - Brian Redban
1,2018-03-19 23:20:58,11990,1093,Owen Benjamin,Joe Rogan,The Joe Rogan Experience,#1093 - Owen Benjamin & Kurt Metzger
2,2018-03-19 23:20:58,11990,1093,Kurt Metzger,Joe Rogan,The Joe Rogan Experience,#1093 - Owen Benjamin & Kurt Metzger
3,2018-03-15 00:26:37,7725,1092,Mary Lynn Rajskub,Joe Rogan,The Joe Rogan Experience,#1092 - Mary Lynn Rajskub
4,2018-03-14 23:39:03,9406,1091,Daniele Bolelli,Joe Rogan,The Joe Rogan Experience,#1091 - Daniele Bolelli
5,2018-03-13 23:22:42,11795,1090,Andrew Santino,Joe Rogan,The Joe Rogan Experience,#1090 - Andrew Santino
6,2018-03-08 02:07:37,13856,1089,John Dudley,Joe Rogan,The Joe Rogan Experience,#1089 - John Dudley
7,2018-03-07 03:06:07,11805,1088,Bryan Callen,Joe Rogan,The Joe Rogan Experience,#1088 - Bryan Callen
8,2018-03-05 23:37:15,10793,1087,Sturgill Simpson,Joe Rogan,The Joe Rogan Experience,#1087 - Sturgill Simpson
9,2018-03-01 00:54:02,11608,1086,Rory Albanese,Joe Rogan,The Joe Rogan Experience,#1086 - Rory Albanese


In [4]:
split_hosts = splitDataFrameList(df, 'hosts', ', ')
split_hosts

G1 = nx.from_pandas_dataframe(split_hosts, 'guests', 'hosts', edge_attr=['date', 'duration', 'podcast'], create_using=nx.MultiDiGraph())

In [57]:

G1.edges(data=True)
#nx.average_shortest_path_length(G1)
#nx.eccentricity(G1,'Joe Rogan')
nx.shortest_path_length(G1, "One Pointed Mind")
#nx.shortest_path(G1, 'President Barack Obama', 'Dan Harmon')
#len(G1.edges())

{'Al Madrigal': 4,
 'Alison Rosen': 4,
 'Andy Cohen': 5,
 'Anna Faris': 5,
 'April Macie': 4,
 'Ari Shaffir': 3,
 'Aubrey Marcus': 2,
 'Bert Kreischer': 4,
 'Big Jay Oakerson': 4,
 'Bill Burr': 4,
 'Bill Maher': 6,
 'Bill Simmons': 4,
 'Bob Forrest': 5,
 'Brandt Tobler': 4,
 'Brendan Schaub': 3,
 'Brendon Walsh': 4,
 'Brian Redban': 4,
 'Brian Rose': 4,
 'Brody Stevens': 4,
 'Bryan Callen': 3,
 'Cameron Esposito': 4,
 'Casey Wilson': 5,
 'Cheryl Strayed': 7,
 'Chris Grosso': 3,
 'Chris Hardwick': 4,
 'Chris Mancini': 5,
 'Chris Ryan': 5,
 'Chris Vernon': 5,
 'Christina Pazsitzky': 4,
 'Christopher Ryan': 3,
 'Conner Moore': 2,
 'Cory Allen': 1,
 'Dan Harmon': 4,
 'Dan Harris': 4,
 'Dan Savage': 5,
 'Dan St. Germain': 5,
 'Danielle Schneider': 5,
 'Dave Asprey': 4,
 'Dave Rubin': 4,
 'Dave Shumka': 5,
 'Dave Smith': 4,
 'David Axelrod': 7,
 'Dax Shepard': 6,
 'Derek Colanduno': 7,
 'Dom Irrera': 4,
 'Doug Benson': 4,
 'Doug Stanhope': 4,
 'Dr. Drew': 5,
 'Duncan Trussell': 3,
 'Echo Cha

In [24]:
raghu_markus = 'rss_files/raghu_markus.xml'

def clean_raghu_markus(raghu_markus):

    raghu_markus = xml_to_df(raghu_markus)
        
    date_parser('\s\-0000', raghu_markus)
    
    raghu_markus['guests'] = raghu_markus.title.str.extract(r'Ep. \d+ (?:-|–) ([\w\s\.\,\'\`\’\&\"\-\‘\/]+)',expand=True)
    
    guest_split(' Part', raghu_markus)
    guest_split(' Returns', raghu_markus)
    guest_split(' Live', raghu_markus)
    guest_split(' Goes', raghu_markus)
    guest_split(' Explores', raghu_markus)
    guest_split("'s ", raghu_markus)
    guest_split_last('with ', raghu_markus)
    guest_split_last('w/ ', raghu_markus)
    guest_split_last('Clearly ', raghu_markus)
    guest_split_last('Founder ', raghu_markus)
    
    replace('Anne and Joan Watts', 'Anne Watts and Joan Watts', raghu_markus)
    replace('It', np.nan, raghu_markus)
    replace('2018', np.nan, raghu_markus)
    replace("Teal Swan's Vision", "Teal Swan", raghu_markus)
    
    
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' of ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' Of ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' the ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'The ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' To ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' Our ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' better ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' Into ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' is ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' with ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' Your ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' in ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' it ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' Days') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' than ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' as ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Politics') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Rascals') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Yourself') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Musings') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'World') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Attention') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Glories') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'After') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Suffering') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Digital') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Close') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Teachers') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'India') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Irregular') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'India') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Spontaneous') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Brain') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Within') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Happiness') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Unconscious') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Conscious') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Street') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Giant') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Matters') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'True') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Mindfulness') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' his ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' to ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' for ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Imagination') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Shock') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Power') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Thought') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r' A ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'G-d') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Mode') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Buddhism') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Abundance') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Quantum') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Tour ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Presence') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Mental') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Home') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'EUDAIMONIA') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'There') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Love') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'One ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Seekers ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Dream ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Sacred ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'Jungle ') == False]
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'What ') == False]
    
    raghu_markus = raghu_markus[pd.notnull(raghu_markus['guests'])]
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ', and ')
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ', & ')
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ', ')
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ',')
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ' & ')
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ' And ')
    raghu_markus = splitDataFrameList(raghu_markus, 'guests', ' and ')
    
    raghu_markus = raghu_markus[raghu_markus.guests.str.contains(r'MLK') == False]

    raghu_markus['guests'] = [g.rstrip() for g in raghu_markus['guests']]
    raghu_markus.replace('', np.nan, inplace=True)
    raghu_markus = raghu_markus[pd.notnull(raghu_markus['guests'])]
    
    return raghu_markus

clean_raghu_markus(raghu_markus)[0:50]

,date,duration,guests,title
0,2018-04-06 22:21:57,4619,Lama Tsultrim Allione,Ep. 234 - Wisdom Rising with Lama Tsultrim All...
1,2018-03-24 14:56:45,3751,Dr. Mark Epstein,Ep. 232 – Getting Over Yourself with Dr. Mark ...
2,2018-03-17 15:10:31,3287,East Forest,Ep. 231 - East Forest
3,2018-03-10 00:13:36,3366,Ethan Nichtern,Ep. 230 - The Road Home with Ethan Nichtern
4,2018-03-02 23:48:16,3408,Roshi Joan Halifax,Ep. 229 - Standing at the Edge with Roshi Joan...
5,2018-02-17 01:05:12,3517,Anne Watts,Ep. 227 – Honoring Alan Watts with Anne and J...
6,2018-02-17 01:05:12,3517,Joan Watts,Ep. 227 – Honoring Alan Watts with Anne and J...
7,2018-02-10 01:55:28,3082,Lama Surya Das,Ep. 226 - Reality and Karma with Lama Surya Das
8,2018-02-03 00:18:49,3856,Sharon Salzberg,Ep. 225 - Getting Real with Sharon Salzberg
9,2018-01-26 17:36:24,4805,Joseph Goldstein,Ep. 224 - The Craving for Becoming with Joseph...


In [ ]:
carasantamaria = xml_to_df('rss_files/carasantamaria.xml')
carasantamaria

In [ ]:
econtalk2013 = xml_to_df_desc('rss_files/econtalk2013.xml')
econtalk2006 = xml_to_df_desc('rss_files/econtalk2006.xml')

def clean_econtalk2006(econtalk2006):
        
    date_parser('\s(EST|GMT)$', econtalk2006)
    
    econtalk2006['guests'] = econtalk2006.description.str.extract(r'([\w\s\'\.]+)',expand=True)
    
    guest_split(', ', econtalk2006)
    guest_split(' of', econtalk2006)
    guest_split(' talks', econtalk2006)
    
    guest_split_last('Nobel Laureate ', econtalk2006)
    guest_split_last('syndicated columnist ', econtalk2006)
    guest_split_last('Investigative journalist ', econtalk2006)
    guest_split_last('Author ', econtalk2006)
    guest_split_last('Novelist ', econtalk2006)
    guest_split_last('The legendary ', econtalk2006)
            
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('This is the') == False]
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('EconTalk host') == False]
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('In this bonus') == False]
    
    econtalk2006 = splitDataFrameList(econtalk2006, 'guests', ' and ')

    return econtalk2006

econtalk2006 = clean_econtalk2006(econtalk2006)

econtalk2006

## podcasts to download

